# Points of Dispense
#### Minimize maximum distance and vary number of PODs
Maxwell Kennady, Nora Murray, Elizabeth Speigle

In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

### Optimization

Read in data from ...

In [2]:
distances = pd.read_csv('data/OD_Pairs_Distances.csv')
population = pd.read_excel('data/BG_master.xlsx')
bg_included = pd.read_csv('data/bg_pop_included.csv')

In [3]:
dist_miles = distances.pivot(index='block_group', columns='pod_id', values='Miles')
dist_time = distances.pivot(index='block_group', columns='pod_id', values='TravelTime')

In [4]:
dist = dist_miles.values                     # can change to dist_time
N = population['population'].values          # can change to num_hhs for households
prop = bg_included['all_included'].values    # can change to random_included or other column

Create indices for block groups and PODs

In [5]:
blocks = range(len(N))
pods = range(len(dist[0]))

Initialize model for POD locations

In [6]:
pods_open = range(10, 48)
pods_open_results = {}

In [7]:
for i in pods_open: 
    ### Initialize model for POD locations
    m = gp.Model('POD_locations')
    
    ### Add decision variables
    x = m.addVars(pods, vtype=GRB.BINARY, name='x')
    y = m.addVars(blocks, pods, vtype=GRB.BINARY, name='y')
    
    ### Set objective to minimize maximum distance
    z = m.addVar(vtype=GRB.CONTINUOUS, name='z')
    m.setObjective(z, GRB.MINIMIZE)
    
    ### Add constraints
    #  y[i,j] can only be 1 if x[i] is also 1, meaning POD i is opened
    m.addConstrs((y[j, i] <= x[i] for i in pods for j in blocks), name='y_if_x')
    
    # each block group must be assigned one shelter
    m.addConstrs((gp.quicksum(y[j, i] for i in pods) == 1
             for j in blocks), name='all_blocks_assigned')
    
    # number of PODs opened must be less than num_pods
    m.addConstr((gp.quicksum(x[i] for i in pods) <= i), name='pods_opened')
    
    # maximum distance
    m.addConstrs((z >= dist[j, i] * y[j, i] for i in pods for j in blocks), 
             name='Distance_Below_Max')
    
    # capacity at each pod
    # m.addConstrs((gp.quicksum(N[j] * x[i] * y[j, i] for j in blocks) <= cap[i]
    #        for i in pods), name='Capacity')
    
    ### Optimize
    print('Solving for:', i, 'PODs')
    m.optimize()
    
    ### Analysis
    block_pod_list = [[j,i] for j in blocks for i in pods if y[j, i].x==1]
    pods_open_results[i] = block_pod_list

Using license file C:\Users\Elizabeth\gurobi.lic
Academic license - for non-commercial use only
Solving for: 10 PODs
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 104501 rows, 51748 columns and 258547 nonzeros
Model fingerprint: 0x1f644705
Variable types: 1 continuous, 51747 integer (51747 binary)
Coefficient statistics:
  Matrix range     [8e-02, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve time: 0.46s
Presolved: 104501 rows, 51748 columns, 258547 nonzeros
Variable types: 1 continuous, 51747 integer (51747 binary)
Found heuristic solution: objective 36.3382164
Found heuristic solution: objective 30.4291814
Found heuristic solution: objective 30.1723360

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 7.133241e-01, 18746 iterations, 2.61 seconds
Total elapsed ti

In [8]:
pods_open_minmax = open('pods_open_minmax','wb')
pickle.dump(pods_open_results, pods_open_minmax)
pods_open_minmax.close()